# Мой "доморощенный" парсер-паук

Парсер сам ищет новую ссылку на текущей странице (начиная с начальной) и переползает на нее. Парсинг продолжается пока не спарсится заданное количество страниц или пока страницы не закончатся.   

Примечание:    
- между страницами можно вставлять (наверное) случайную задержку для иммитации естественного посещения страницы
- при ошибке запроса - запрос на этот url повторяется

In [81]:
# import scrapy
import csv
import pandas as pd
import numpy as np
import requests
import bs4
from tqdm import tqdm_notebook

In [156]:
# class TorgmailSpider(scrapy.Spider):
class TorgmailSpider():
    '''
    start_url - начальный адрес
    site - узел, с которого мы парсим данные
    Методы:
    .parse() - метод для парсинга
    '''
    
    name = "torgmail_spider"
    def __init__(self, start_url = 'https://torg.mail.ru/review/goods/mobilephones/', 
                site = 'https://torg.mail.ru') :
        self.start_url = start_url
        self.start_urls = [start_url]
        self.site = site
    
    def parse_page(self, url, path_and_filename):
        with open(path_and_filename, 'a') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter='\t')

                # если возникает ошибка (страница не найдена (404) или запрос прерван (443) или еще что-то),
                # то генерируется исключение и печатается сообщение об этом, но код не прерывается
            
            cycle_flag = True                     # организуем повтор запроса при сбое загрузки страницы
            while cycle_flag:
                try:                              # обрабатываем возможные исключения
                    cycle_flag = False
                    request = requests.get(url)
                    request.raise_for_status()
                except requests.exceptions.HTTPError as err:
    #                     print('Oops. HTTP Error occured. Response is: {content} for url {url}'
    #                           .format(content=err.response, url=url))
        #                 print('Response is: {content}'.format(content=err.response.content))
                    print('Oops. HTTP Error occured: %s' % err)
                    cycle_flag = True
                    continue
                except requests.exceptions.ConnectionError as err1:
        #                 print('ERROR: %s' % err1.args[0])
                    print('Oops. Connection Error occured. Response is: {content} for url {url}'.format(content=err1.response, 
                                                                                                        url=url))
                    cycle_flag = True
                    continue
                except requests.exceptions.RequestException as err2:
        #                 print('ERROR: %s' % err2.args[0])
                    print('Oops. RequestException Error occured. Response is: {content} for url {url}'
                          .format(content=err2.response, 
                                                                                                        url=url))
                    cycle_flag = True
                    continue
                else:
                    text = request.text
                    parser = bs4.BeautifulSoup(text, 'lxml')    

                    for item in  parser.find_all('div', attrs={'class':'review-item'}):

                        # \t - заменяем на пробелы, так как табуляцию используем как разделитель (запятые и так в тексте есть)
                        review = item.find('a', attrs={'class':'more'}).attrs['full-text'].replace('\t', ' ') \
                                        if item.find('a', attrs={'class':'more'}) \
                                        else item.find('p', attrs={'class':"review-item__paragraph"}).text.replace('\t', ' ')
                        rating = float(item.find('span', attrs={'class':"review-item__rating-counter"}).text.replace(',', '.'))

                        try:
                            csvwriter.writerow([review, rating])
                        except Exception as err3:
                            print('Oops. Exception Error occured for url {url}. Response is: {content}'.format(url=url, 
                                                                                                               content=err3))    

                    # формируем новую ссылку для "переползания" туда "паука"
                    parse_for_next_page = parser.find('ul', attrs={'class':'pager-list js-pager'}
                                                     ).find('a', attrs={'rel':'next'})
                    if parse_for_next_page is not None:
                        next_page = self.site + parse_for_next_page.attrs['href']
                    else:
                        next_page = False

        return next_page

    
    def parse(self, num_pages=3, path_and_filename='tmp.csv'):
        '''
        num_pages: int или None 
                - число страниц для парсинга. Если None - страницы парсятся пока все не закончатся
        path_and_filename: str 
                - путь и файл, куда сохраняем спарсенные данные
        '''
        with open(path_and_filename, 'w') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter='\t')
            csvwriter.writerow(['review', 'rating']) # пишем заголовки колонок
            
            next_page = self.start_url
            if num_pages is not None:
                for n in tqdm_notebook(range(num_pages)):

                    # сюда можно вставить случайную задержку для иммитации естественного посещения страницы
                    next_page = self.parse_page(next_page, path_and_filename)
                    if not next_page:
                        print('\nСтраницы для парсинга закончились')
                        break
            else:
                while_flag = True
                while while_flag:
                    first_str = True
                    if first_str:
                        print('Парсинг первых 500 страниц:')
                        first_str = False
                    else:
                        print('парсинг следующих 500 страниц:')
                    for l in tqdm_notebook(range(500)):
                        next_page = self.parse_page(next_page, path_and_filename)
                        if not next_page:
                            print('\nВсе страницы для парсинга закончились')
                            while_flag = False
                            break
        pass

#### тестируем: тест 1

In [157]:
%%time
torgmailSpider = TorgmailSpider('https://torg.mail.ru/review/goods/mobilephones/?page=920')
torgmailSpider.parse(None, 'data/tmp.csv') # парсим 5 страниц

Парсинг первых 500 страниц:


A Jupyter Widget

Oops. Connection Error occured. Response is: None for url https://torg.mail.ru/review/goods/mobilephones/?page=920

Все страницы для парсинга закончились

Wall time: 5.51 s


#### тестируем: тест 2

In [158]:
%%time
torgmailSpider = TorgmailSpider('https://torg.mail.ru/review/goods/mobilephones/?page=920')
torgmailSpider.parse(10, 'data/tmp.csv') # парсим 5 страниц

A Jupyter Widget


Страницы для парсинга закончились

Wall time: 3.3 s


In [159]:
tmp_df_by_csv = pd.read_csv('data/tmp.csv', sep='\t', index_col=None, header=0, encoding='cp1251')

print(tmp_df_by_csv.shape)
tmp_df_by_csv.head()

(60, 2)


,review,rating
0,настроек захода в интернет через комп. МТС пор...,4.0
1,\r\nЭтот телефон просто супер. Некоторые говор...,5.0
2,\r\nНа мой взляд это самый подходящей тел. для...,4.0
3,"\r\nНедавно купила данный агрегат... Чтож, тел...",5.0
4,"Очень хороший телефон....., качество сборки на...",5.0


**ПРИМЕЧАНИЕ по поводу Scrapy**:   
Если класс будет наследником **scrapy.Spider**, то кроме **определенных мною** методов добавляется еще куча методов, которые **уже** определены в scrapy.Spider и которые делают кучу разных полезных и хороших вещей, в том числе
- запросы и поиск нужных элементов на странице на базе селекторов **CSS** и **XPath** и **встроенных** средств для запросов
- плюс еще есть разные дополнительные возможности и настройки (работа с капчей, задержки при посещении страниц и т.д.)

В моем же "доморощенном" пауке я это делаю вручную на базе библиотек **requests** и **BeautifulSoup** (вместо CSS и XPath) и (соответственно) **без** дополнительных фичей